In [6]:
import cv2
import numpy as np

In [8]:
cap = cv2.VideoCapture(1)

classesFile = 'obj.names'
classNames = []

with open(classesFile, 'rt') as f:
    classNames = f.read().rstrip('\n').split('\n')
print(classNames)
print(len(classNames))

while True:
    succes, img = cap.read()
    
    cv2.imshow('Image' ,img)
    cv2.waitKey(1)

['Expiration date']
1


error: OpenCV(4.5.1) C:\Users\appveyor\AppData\Local\Temp\1\pip-req-build-kh7iq4w7\opencv\modules\highgui\src\window.cpp:376: error: (-215:Assertion failed) size.width>0 && size.height>0 in function 'cv::imshow'
